Inspired by a wonderful Pluralsight learning Getting Started with Apache Spark on Databricks of Janavi Ravi, I have made a number of transformations using the data taken from https://www.kaggle.com/datasets/yaranathakur/largest-companies-in-the-united-states-by-revenue. The data concerns the largest companies in the United States by revenue, based on the 2022 Fortune 500 rankings.

reading the csv file and writing it to the data frame

In [0]:
mydf = spark.read.format('csv')\
        .option("inferSchema", "True")\
        .option("header", "True")\
        .option("sep", ",")\
        .load("dbfs:/FileStore/files/LargestCompaniesInUSAbyReveneue.csv")

showing how the data frame looks like

In [0]:
mydf.show()

+----+--------------------+--------------------+----------------------+--------------+---------+--------------------+
|Rank|                Name|            Industry|Revenue (USD millions)|Revenue growth|Employees|        Headquarters|
+----+--------------------+--------------------+----------------------+--------------+---------+--------------------+
|   1|             Walmart|General merchandi...|               572,754|          2.4%|2,300,000|Bentonville, Arka...|
|   2|              Amazon|Retail and Cloud ...|               469,822|         21.7%|1,608,000| Seattle, Washington|
|   3|               Apple|Electronics industry|               365,817|         33.2%|  154,000|Cupertino, Califo...|
|   4|          CVS Health|          Healthcare|               292,111|         32.0%|  258,000|Woonsocket, Rhode...|
|   5|  UnitedHealth Group|          Healthcare|               287,597|         11.8%|  350,000|Minnetonka, Minne...|
|   6|         Exxon Mobil|  Petroleum industry|        

verifying the schema data types (to use some data to make calculations, data types should be numeric)

In [0]:
mydf.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Revenue (USD millions): string (nullable = true)
 |-- Revenue growth: string (nullable = true)
 |-- Employees: string (nullable = true)
 |-- Headquarters: string (nullable = true)



converting "Revenue (USD millions)", "Revenue growth" and "Employees" column data from string to integer

In [0]:
from pyspark.sql.functions import regexp_replace

In [0]:
mydf = mydf.withColumn("Revenue (USD millions)",regexp_replace("Revenue (USD millions)", ",","").cast('int'))

In [0]:
mydf.show()

+----+--------------------+--------------------+----------------------+--------------+---------+--------------------+
|Rank|                Name|            Industry|Revenue (USD millions)|Revenue growth|Employees|        Headquarters|
+----+--------------------+--------------------+----------------------+--------------+---------+--------------------+
|   1|             Walmart|General merchandi...|                572754|          2.4%|2,300,000|Bentonville, Arka...|
|   2|              Amazon|Retail and Cloud ...|                469822|         21.7%|1,608,000| Seattle, Washington|
|   3|               Apple|Electronics industry|                365817|         33.2%|  154,000|Cupertino, Califo...|
|   4|          CVS Health|          Healthcare|                292111|         32.0%|  258,000|Woonsocket, Rhode...|
|   5|  UnitedHealth Group|          Healthcare|                287597|         11.8%|  350,000|Minnetonka, Minne...|
|   6|         Exxon Mobil|  Petroleum industry|        

In [0]:
mydf = mydf.withColumn("Revenue growth",regexp_replace("Revenue growth", "%","").cast('double'))

In [0]:
mydf.show()

+----+--------------------+--------------------+----------------------+--------------+---------+--------------------+
|Rank|                Name|            Industry|Revenue (USD millions)|Revenue growth|Employees|        Headquarters|
+----+--------------------+--------------------+----------------------+--------------+---------+--------------------+
|   1|             Walmart|General merchandi...|                572754|           2.4|2,300,000|Bentonville, Arka...|
|   2|              Amazon|Retail and Cloud ...|                469822|          21.7|1,608,000| Seattle, Washington|
|   3|               Apple|Electronics industry|                365817|          33.2|  154,000|Cupertino, Califo...|
|   4|          CVS Health|          Healthcare|                292111|          32.0|  258,000|Woonsocket, Rhode...|
|   5|  UnitedHealth Group|          Healthcare|                287597|          11.8|  350,000|Minnetonka, Minne...|
|   6|         Exxon Mobil|  Petroleum industry|        

In [0]:
mydf = mydf.withColumn("Employees",regexp_replace("Employees", ",","").cast('int'))

In [0]:
mydf.show()

+----+--------------------+--------------------+----------------------+--------------+---------+--------------------+
|Rank|                Name|            Industry|Revenue (USD millions)|Revenue growth|Employees|        Headquarters|
+----+--------------------+--------------------+----------------------+--------------+---------+--------------------+
|   1|             Walmart|General merchandi...|                572754|           2.4|  2300000|Bentonville, Arka...|
|   2|              Amazon|Retail and Cloud ...|                469822|          21.7|  1608000| Seattle, Washington|
|   3|               Apple|Electronics industry|                365817|          33.2|   154000|Cupertino, Califo...|
|   4|          CVS Health|          Healthcare|                292111|          32.0|   258000|Woonsocket, Rhode...|
|   5|  UnitedHealth Group|          Healthcare|                287597|          11.8|   350000|Minnetonka, Minne...|
|   6|         Exxon Mobil|  Petroleum industry|        

verifying if some data types are ready to calculate

In [0]:
mydf.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Revenue (USD millions): integer (nullable = true)
 |-- Revenue growth: double (nullable = true)
 |-- Employees: integer (nullable = true)
 |-- Headquarters: string (nullable = true)



showing the companies with the revenue > 200000

In [0]:
mydf.select('Name', 'Revenue (USD millions)').where(mydf['Revenue (USD millions)'] > 200000).show()

+--------------------+----------------------+
|                Name|Revenue (USD millions)|
+--------------------+----------------------+
|             Walmart|                572754|
|              Amazon|                469822|
|               Apple|                365817|
|          CVS Health|                292111|
|  UnitedHealth Group|                287597|
|         Exxon Mobil|                285640|
|  Berkshire Hathaway|                276094|
|            Alphabet|                257637|
|McKesson Corporation|                238228|
|   AmerisourceBergen|                213989|
+--------------------+----------------------+



showing the companies specializing in technology

In [0]:
mydf.select('Name', 'Revenue (USD millions)', 'Headquarters').filter(mydf['Industry'].isin(['Technology'])).show()

+-----------------+----------------------+--------------------+
|             Name|Revenue (USD millions)|        Headquarters|
+-----------------+----------------------+--------------------+
|         Alphabet|                257637|Mountain View, Ca...|
|        Microsoft|                198087| Redmond, Washington|
|   Meta Platforms|                117385|Menlo Park, Calif...|
|Dell Technologies|                106995|   Round Rock, Texas|
|            Intel|                 79024|Santa Clara, Cali...|
|              IBM|                 72344|    Armonk, New York|
|               HP|                 63487|Palo Alto, Califo...|
+-----------------+----------------------+--------------------+



showing the number of companies by industry

In [0]:
mydf.groupBy('Industry').count().show()

+--------------------+-----+
|            Industry|count|
+--------------------+-----+
|General merchandi...|    1|
|Aerospace and Def...|    1|
|           Insurance|    7|
|Automotive and En...|    1|
|          Healthcare|    6|
|  Financial services|    1|
|           Machinery|    1|
|              Metals|    1|
|Consumer products...|    1|
|              Health|    1|
|Telecom Hardware ...|    1|
|             Apparel|    1|
|Airspace and defense|    1|
|               Media|    1|
|  Petroleum industry|    9|
|       Food industry|    1|
|          Automotive|    1|
|        Conglomerate|    4|
|Agriculture coope...|    1|
|           Beverages|    1|
+--------------------+-----+
only showing top 20 rows



showing the companies revenue grouped by industry

In [0]:
mydf.groupBy('Industry').agg({'Revenue (USD millions)':'sum'}).show()

+--------------------+---------------------------+
|            Industry|sum(Revenue (USD millions))|
+--------------------+---------------------------+
|General merchandi...|                     572754|
|Aerospace and Def...|                      67044|
|           Insurance|                     329663|
|Automotive and En...|                      53823|
|          Healthcare|                    1065548|
|  Financial services|                     127202|
|           Machinery|                      50971|
|              Metals|                      36484|
|Consumer products...|                      76118|
|              Health|                     238228|
|Telecom Hardware ...|                      49818|
|             Apparel|                      44538|
|Airspace and defense|                      38469|
|               Media|                      67418|
|  Petroleum industry|                    1010973|
|       Food industry|                      85959|
|          Automotive|         

showing the top ten companies by revenue

In [0]:
mydf_aggr = mydf.groupBy('Industry').agg({'Revenue (USD millions)':'sum'})


In [0]:
mydf_aggr.printSchema()

root
 |-- Industry: string (nullable = true)
 |-- sum(Revenue (USD millions)): long (nullable = true)



In [0]:
from pyspark.sql.functions import col, desc

In [0]:
mydf_aggr10 = mydf_aggr.orderBy(col('sum(Revenue (USD millions))').desc()).limit(10)

In [0]:
mydf_aggr10.show()

+--------------------+---------------------------+
|            Industry|sum(Revenue (USD millions))|
+--------------------+---------------------------+
|          Healthcare|                    1065548|
|  Petroleum industry|                    1010973|
|              Retail|                     905624|
|          Technology|                     894959|
|          Financials|                     854099|
|Pharmaceutical in...|                     691429|
|        Conglomerate|                     583542|
|General merchandi...|                     572754|
|Retail and Cloud ...|                     469822|
|Electronics industry|                     365817|
+--------------------+---------------------------+



In [0]:
display(mydf_aggr10)

Industry,sum(Revenue (USD millions))
Healthcare,1065548
Petroleum industry,1010973
Retail,905624
Technology,894959
Financials,854099
Pharmaceutical industry,691429
Conglomerate,583542
General merchandisers,572754
Retail and Cloud Computing,469822
Electronics industry,365817


showing the top ten companies by Revenue growth

In [0]:
mydf_aggr2 = mydf.groupBy('Industry').agg({'Revenue growth':'sum'})

In [0]:
mydf_aggr2.show()

+--------------------+-------------------+
|            Industry|sum(Revenue growth)|
+--------------------+-------------------+
|General merchandi...|                2.4|
|Aerospace and Def...|                2.5|
|           Insurance|  80.69999999999999|
|Automotive and En...|               70.7|
|          Healthcare|               91.2|
|  Financial services|                1.8|
|           Machinery|               22.1|
|              Metals|               81.2|
|Consumer products...|                7.3|
|              Health|                3.1|
|Telecom Hardware ...|                1.0|
|             Apparel|               19.1|
|Airspace and defense|                8.7|
|               Media|                3.1|
|  Petroleum industry|              698.0|
|       Food industry|               32.5|
|          Automotive|                3.7|
|        Conglomerate|               34.8|
|Agriculture coope...|                1.4|
|           Beverages|               17.1|
+----------

In [0]:
mydf_aggr10_revenue_growth = mydf_aggr2.orderBy(col('sum(Revenue growth)').desc()).limit(10)

In [0]:
mydf_aggr10_revenue_growth.show()

+--------------------+-------------------+
|            Industry|sum(Revenue growth)|
+--------------------+-------------------+
|  Petroleum industry|              698.0|
|Pharmaceutical in...| 165.29999999999998|
|              Retail|              136.0|
|          Technology|              124.8|
|          Financials|  96.00000000000001|
|          Healthcare|               91.2|
|              Metals|               81.2|
|           Insurance|  80.69999999999999|
|Automotive and En...|               70.7|
|   Chemical industry|               42.6|
+--------------------+-------------------+



showing the number of employees by state (top 10)

In [0]:
from pyspark.sql.functions import split

In [0]:
mydf = mydf.withColumn('City', split(mydf['Headquarters'], ',').getItem(0)) \
       .withColumn('State', split(mydf['Headquarters'], ',').getItem(1))

In [0]:
mydf.show()

+----+--------------------+--------------------+----------------------+--------------+---------+--------------------+-------------+-------------+
|Rank|                Name|            Industry|Revenue (USD millions)|Revenue growth|Employees|        Headquarters|         City|        State|
+----+--------------------+--------------------+----------------------+--------------+---------+--------------------+-------------+-------------+
|   1|             Walmart|General merchandi...|                572754|           2.4|  2300000|Bentonville, Arka...|  Bentonville|     Arkansas|
|   2|              Amazon|Retail and Cloud ...|                469822|          21.7|  1608000| Seattle, Washington|      Seattle|   Washington|
|   3|               Apple|Electronics industry|                365817|          33.2|   154000|Cupertino, Califo...|    Cupertino|   California|
|   4|          CVS Health|          Healthcare|                292111|          32.0|   258000|Woonsocket, Rhode...|   Woon

In [0]:
mydf_aggr3 = mydf.groupBy('State').agg({'Employees':'sum'})

In [0]:
mydf_aggr3.show()

+-------------+--------------+
|        State|sum(Employees)|
+-------------+--------------+
|     Missouri|         72500|
| Pennsylvania|        326000|
|         Ohio|        658738|
|     Illinois|        840315|
|     Arkansas|       2437000|
|   California|       1095013|
| Rhode Island|        258000|
|     Virginia|        110401|
|      Georgia|        970545|
|    Minnesota|        891316|
|     Kentucky|         95500|
|     Michigan|        375700|
|    Tennessee|        728000|
|   New Jersey|        250116|
|    Wisconsin|          7585|
|     New York|       1623158|
|   Washington|       2077000|
|     Nebraska|        372000|
|       Oregon|         73300|
|         D.C.|          7400|
+-------------+--------------+
only showing top 20 rows



In [0]:
mydf_aggr10_employees = mydf_aggr3.orderBy(col('sum(Employees)').desc()).limit(10)

In [0]:
mydf_aggr10_employees.show()

+--------------+--------------+
|         State|sum(Employees)|
+--------------+--------------+
|      Arkansas|       2437000|
|    Washington|       2077000|
|      New York|       1623158|
|    California|       1095013|
|       Georgia|        970545|
| Massachusetts|        966000|
|     Minnesota|        891316|
|         Texas|        849708|
|      Illinois|        840315|
|     Tennessee|        728000|
+--------------+--------------+

